### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set client
client = OpenAI()

In [3]:
# Load dataframe
cleaned_nl_simlex = pd.read_csv("../../../data/dataset/cleaned-nl-simlex-999.csv")

# Remove first three word pairs
cleaned_nl_simlex = cleaned_nl_simlex.iloc[331:664]

# Select subset
# cleaned_nl_simlex = cleaned_nl_simlex.head(150)

# Convert to tuple
tuples_list = list(zip(cleaned_nl_simlex['word1'], cleaned_nl_simlex['word2']))

In [4]:
# Show results
tuples_list

[('spel', 'plezier'),
 ('weekend', 'week'),
 ('koppel', 'paar'),
 ('vrouw', 'echtgenote'),
 ('schaap', 'vee'),
 ('portemonnee', 'tas'),
 ('dak', 'kathedraal'),
 ('boon', 'koffie'),
 ('hout', 'papier'),
 ('bovenkant', 'zijkant'),
 ('misdaad', 'fraude'),
 ('pijn', 'schade'),
 ('minnaar', 'partner'),
 ('avond', 'schemering'),
 ('vader', 'dochter'),
 ('wijn', 'likeur'),
 ('koe', 'geit'),
 ('geloof', 'mening'),
 ('werkelijkheid', 'illusie'),
 ('overeenkomst', 'voorwaarde'),
 ('rijkdom', 'armoede'),
 ('ongeval', 'noodgeval'),
 ('strijd', 'verovering'),
 ('vriend', 'leraar'),
 ('ziekte', 'besmetting'),
 ('spel', 'truc'),
 ('broer', 'zoon'),
 ('tante', 'neef'),
 ('arbeider', 'monteur'),
 ('dokter', 'orthodontist'),
 ('eikenboom', 'esdoorn'),
 ('bij', 'koningin'),
 ('auto', 'fiets'),
 ('doelwit', 'zoektocht'),
 ('augustus', 'maand'),
 ('leger', 'ploeg'),
 ('wolk', 'weer'),
 ('arts', 'dokter'),
 ('ravijn', 'vallei'),
 ('rivier', 'vallei'),
 ('zon', 'lucht'),
 ('doel', 'pijl'),
 ('chocolade', 'ta

### **1. Define and Evaluate Parameters**

In [12]:
# Define prompt
prompt = ("Beoordeel de semantische gelijkenis van het woordpaar: [('{word1}'), ('{word2}')] op een schaal van 0 tot 10, "
          "waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. "
          "Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>)]. "
          "Geef geen extra uitleg of context.")

In [13]:
# Define model
model = "gpt-3.5-turbo-0125"

# Set sample size
# sample_size = 5
sample_size = 15

# Delay between individual API calls
delay = 3.0

# Define number of sublists
# n_sublists = 997
n_sublists = 333

In [14]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

333


In [15]:
# Print prompts for each chunk
print_prompts_single(chunks, sample_size, prompt)

Beoordeel de semantische gelijkenis van het woordpaar: [('spel'), ('plezier')] op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>)]. Geef geen extra uitleg of context.
Beoordeel de semantische gelijkenis van het woordpaar: [('spel'), ('plezier')] op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>)]. Geef geen extra uitleg of context.
Beoordeel de semantische gelijkenis van het woordpaar: [('spel'), ('plezier')] op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>)]. 

In [16]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken_single(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [108, 107, 108, 110, 109, 108, 110, 109, 109, 111, 109, 108, 108, 109, 108, 108, 108, 109, 110, 111, 109, 110, 110, 108, 109, 108, 108, 108, 109, 110, 110, 108, 107, 110, 109, 109, 108, 108, 109, 108, 107, 108, 109, 111, 108, 108, 109, 108, 111, 109, 106, 107, 110, 109, 109, 108, 109, 108, 107, 109, 107, 109, 108, 108, 109, 109, 111, 109, 106, 107, 107, 109, 108, 111, 108, 107, 108, 108, 108, 110, 107, 109, 106, 108, 110, 110, 110, 108, 106, 106, 108, 111, 109, 107, 109, 109, 113, 111, 107, 110, 107, 107, 109, 108, 109, 111, 109, 107, 107, 111, 108, 107, 107, 108, 108, 109, 110, 109, 107, 107, 110, 110, 107, 107, 107, 108, 110, 108, 110, 107, 108, 108, 108, 106, 107, 109, 108, 107, 110, 106, 107, 109, 109, 109, 109, 112, 110, 109, 108, 108, 110, 109, 109, 108, 110, 109, 107, 108, 108, 107, 108, 109, 109, 110, 109, 108, 109, 109, 111, 107, 109, 111, 109, 109, 108, 110, 109, 107, 109, 109, 109, 109, 110, 107, 109, 109, 107, 111, 112, 108, 109, 108,

In [17]:
# Max RPD = 10.000
len(token_counts*15)

4995

In [18]:
# Number of total tokens
sum(token_counts)

36179

### **2. Extract and Process Data**

In [19]:
# Get results from API
# response = get_responses_single(prompt, chunks, model, sample_size, delay)

Processing: 100%|██████████| 4995/4995 [9:43:47<00:00,  7.01s/chunk]   

Total time taken: 35027.72 seconds


In [20]:
# Define filepath
file_path = '../../../data/response/nl/gpt-3.5-turbo-0125/f9-2.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [21]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('uithoudingsvermogen', 'kracht'): [1.2, 2.5, 2.5, 3.56, 0.1, 2.5, 2.5, 0.3, 2.5, 4.25, 3.5, 3.5, 5.2, 3.5]}


In [22]:
# Print duplicate word pairs
print_duplicate_word_pairs(cleaned_nl_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [23]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,spel,plezier,0.20,2.45,0.25,0.15,3.20,0.12,0.35,0.10,0.10,0.15,0.25,0.20,2.56,3.20,1.20
1,weekend,week,8.50,7.50,6.50,6.50,0.20,7.50,7.50,8.50,6.00,7.00,8.00,7.00,8.50,8.50,7.00
2,koppel,paar,8.00,7.50,9.50,9.50,8.50,6.00,6.50,8.00,8.00,8.50,9.00,8.50,8.50,8.75,8.50
3,vrouw,echtgenote,9.80,8.70,9.50,9.50,9.75,9.50,9.50,9.60,9.50,9.50,9.50,9.50,9.75,9.50,8.75
4,schaap,vee,5.25,7.50,6.50,6.75,7.50,7.00,7.25,7.50,6.25,6.25,7.20,6.25,7.50,5.65,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,brief,paragraaf,2.50,0.10,0.15,0.10,0.00,3.20,0.10,0.10,1.00,0.10,0.00,0.29,2.00,1.42,0.00
329,pagina,paragraaf,2.50,1.20,0.20,2.50,0.10,6.25,0.40,2.50,2.50,3.20,0.21,2.50,2.50,1.25,2.00
330,kerktoren,kapel,5.00,6.50,6.25,6.50,7.50,7.50,7.50,6.50,6.50,6.50,7.50,6.50,6.50,5.00,6.50
331,spier,botten,2.20,1.23,3.20,2.50,2.56,5.72,2.30,4.20,0.12,1.20,0.45,3.50,2.15,1.23,0.12


In [27]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    1
dtype: int64


In [28]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
96,uithoudingsvermogen,kracht,1.2,2.5,2.5,3.56,0.1,2.5,2.5,0.3,2.5,4.25,3.5,3.5,5.2,3.5,NaN


In [29]:
# Custom prompt
custom_prompt = ("Beoordeel de semantische gelijkenis van het woordpaar: [('uithoudingsvermogen'), ('kracht')] op een schaal van 0 tot 10, "
                 "waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. "
                 "Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [('woord1', 'woord2', <score>)]. "
                 "Geef geen extra uitleg of context.")


# Make API call
# messages = [{"role": "user", "content": custom_prompt}]
# completion = client.chat.completions.create(
#     model=model,
#     messages=messages,
#     n=1)

# Show results
print(completion.choices[0].message.content)

[('uithoudingsvermogen', 'kracht', 2.50)]


In [30]:
# Manually fix inconsistencies
df.loc[(df['word1'] == 'uithoudingsvermogen') & (df['word2'] == 'kracht'), 'similarity_score_15'] = 2.50

# Show results
df.loc[(df['word1'] == 'uithoudingsvermogen') & (df['word2'] == 'kracht')]

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
96,uithoudingsvermogen,kracht,1.2,2.5,2.5,3.56,0.1,2.5,2.5,0.3,2.5,4.25,3.5,3.5,5.2,3.5,2.5


In [31]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [32]:
# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,spel,plezier,0.20,2.45,0.25,0.15,3.20,0.12,0.35,0.10,0.10,0.15,0.25,0.20,2.56,3.20,1.20
1,weekend,week,8.50,7.50,6.50,6.50,0.20,7.50,7.50,8.50,6.00,7.00,8.00,7.00,8.50,8.50,7.00
2,koppel,paar,8.00,7.50,9.50,9.50,8.50,6.00,6.50,8.00,8.00,8.50,9.00,8.50,8.50,8.75,8.50
3,vrouw,echtgenote,9.80,8.70,9.50,9.50,9.75,9.50,9.50,9.60,9.50,9.50,9.50,9.50,9.75,9.50,8.75
4,schaap,vee,5.25,7.50,6.50,6.75,7.50,7.00,7.25,7.50,6.25,6.25,7.20,6.25,7.50,5.65,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,brief,paragraaf,2.50,0.10,0.15,0.10,0.00,3.20,0.10,0.10,1.00,0.10,0.00,0.29,2.00,1.42,0.00
329,pagina,paragraaf,2.50,1.20,0.20,2.50,0.10,6.25,0.40,2.50,2.50,3.20,0.21,2.50,2.50,1.25,2.00
330,kerktoren,kapel,5.00,6.50,6.25,6.50,7.50,7.50,7.50,6.50,6.50,6.50,7.50,6.50,6.50,5.00,6.50
331,spier,botten,2.20,1.23,3.20,2.50,2.56,5.72,2.30,4.20,0.12,1.20,0.45,3.50,2.15,1.23,0.12


In [33]:
# Define file_path
file_path = '../../../data/prompt/nl/gpt-3.5-turbo-0125/f9-2.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
